In [132]:
import numpy as np
import pandas as pd
import math

In [133]:
# Reading the dataset using Pandas
df = pd.read_csv("LBW_Dataset.csv")

# Normalization & Scaling Functions using Numpy & Pandas

# Outlier Scaling using .quantile() Pandas methods
def scale_outlier(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    min_bound = Q1 - 1.5*IQR
    max_bound = Q3 + 1.5*IQR
    df[column] = np.where(df[column] > max_bound, max_bound, df[column])
    df[column] = np.where(df[column] < min_bound, min_bound, df[column])

# Min-Max Scaling using .min() and .max() Pandas methods
def min_max_scaling(df):    
    df_norm = df.copy()
    for column in df_norm.columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())        
    return df_norm

In [134]:
# Data Preprocessing

# Drop the columns Delivery Phase(1: 90, 2: 2, NaN: 4) and Education(5: 93, NaN: 3)
df = df.drop(["Delivery phase", "Education", "Community"], axis = 1)

# Replacing Nan of Weights with the Mean of its respective Result category
mean_0 = (df.loc[df['Result'] == 0])['Weight'].mean()
mean_1 = (df.loc[df['Result'] == 1])['Weight'].mean()

df["Weight"] = np.where((df["Result"] == 0) & (df["Weight"].isna()), mean_0, df["Weight"])
df["Weight"] = np.where((df["Result"] == 1) & (df["Weight"].isna()), mean_1, df["Weight"])

# For now, Filling Numeric Columned NaN Values with Mean
df["Age"] = df["Age"].fillna(df["Age"].mean())
df["HB"] = df["HB"].fillna(df["HB"].mean())
df["BP"] = df["BP"].fillna(df["BP"].mean())

# Very Basic Method of taking care of Outliers(Replace with IQR, Min-Max) for Age & BP columns
scale_outlier(df, "Age")
scale_outlier(df, "BP")

# Labelling Residence = 2 as Residence = 0 to get Binary Labelled Column (Before: Residence(1,2), After: Residence(1,0))
df["Residence"] = np.where(df["Residence"] == 2, 0, df["Residence"])
# Filling NaN with Mode = 1
df["Residence"] = df["Residence"].fillna(1)

# Converting IFA(int) to IFA(float)
df["IFA"] = df["IFA"].astype(float)

# Moving converted Float Result, to get it as the last Column
res = df["Result"].astype(float)
df = df.drop(["Result"], axis = 1)
df["Result"] = res

# Performing Normalization of the dataset (into ranges from 0 to 1) using Pandas
df = min_max_scaling(df)

In [278]:
# Creating Train-Test Splits of the dataset using .train_test_split() in Sklearn
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1].values
y = df.iloc[:,-1:].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [293]:
def sigmoid(x):
  return [1 / (1 + math.exp(-ele)) for ele in x ]

class layer():
    def __init__(self, input_units, output_units, alpha = 0.00001, activation = 'tanh'):
        
        self.activation = activation
        self.weights = np.random.normal(loc=0.0, 
                                        scale = np.sqrt(2/(input_units+output_units)), 
                                        size = (input_units,output_units))
        self.bias = np.zeros(output_units)
        self.input = np.zeros(output_units)
        self.activated_output = np.zeros(output_units)
        
        # adam optimiser : parameters
        self.alpha = alpha
        self.t = 0
        self.m = 0
        self.v = 0;
        self.beta_1 = 0.9
        self.beta_2 = 0.99
        self.epsilon = 1e-8
        
    def forward_prop(self, inputs):
        self.input = inputs
        forward_units = np.dot(inputs, self.weights) + self.bias
        
        if self.activation == 'tanh':
            self.activated_output = np.tanh(forward_units)
            
        elif self.activation == 'relu':
            self.activated_output = np.maximum(0, forward_units)
            
        elif self.activation == 'logistic':
            self.activated_output = sigmoid(forward_units)
            
        elif self.activation == 'identity':
            self.activated_output = forward_units
        
        return self.activated_output
    
    def update_weights(self, grad):
        # adam optimiser
        grad = grad.reshape(self.weights.shape)
        self.m = self.beta_1*self.m + grad*(1-self.beta_1)
        self.v = self.beta_2*self.v + np.square(grad)*(1-self.beta_2)
        self.t += 1
        
        m_hat = self.m/(1 - pow(self.beta_1, self.t))
        v_hat = self.v/(1 - pow(self.beta_2, self.t))
        tmp = self.alpha*(m_hat/(v_hat + self.epsilon))
                          
        self.weights = self.weights - tmp           
    
    def loss_function(self, target):
        return [-(target*math.log(ele) + (1-target)*math.log(1-ele)) for ele in self.activated_output]
        

In [294]:
input_units = 6
output_units = 1
hidden_layer1_units = 20

classifier = []
classifier.append(layer(input_units, hidden_layer1_units))
classifier.append(layer(hidden_layer1_units, output_units, activation = 'logistic'))

In [297]:
for ind, inputs in enumerate(X_train):
    inputs = X_train[ind]
    outputs = y_train[ind]
    for layer in classifier:
        inputs = layer.forward_prop(inputs)

    prediction = inputs

    output_layer = classifier[1]
    grad = gradient_sigmoid(prediction, output_layer.input, outputs)    
    output_layer.update_weights(grad)
    
#     hidden_layer = classifier[0]
#     grad = gradient_tanh()
#     hidden_layer.update_weights(grad)
    

In [195]:
def gradient_sigmoid(z, A, y):
    grad = (z-y)*A
    return grad

In [196]:
def gradient_tanh():
    # have to complete
    pass

In [300]:
for ind, inputs in enumerate(X_train):
    outputs = y_train[ind]
    for layer in classifier:
        inputs = layer.forward_prop(inputs)
        
    prediction = inputs
#     print(prediction, outputs)